In [6]:
import shutil
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas_path import path
from pathlib import Path
import pytorch_lightning as pl
import torch

import albumentations as A

from cloud_model38 import CloudModel

import warnings
warnings.filterwarnings("ignore")

DATA_DIR = Path.cwd().parent.resolve() / "CloudCover/data"
TRAIN_FEATURES = DATA_DIR / "data/train_features"
TRAIN_LABELS = DATA_DIR / "data/train_labels"

BANDS = ["B02", "B03", "B04", "B08"]

In [7]:
train_meta = pd.read_csv('train_meta_clear.csv')

In [8]:
random.seed(9)  # set a seed for reproducibility
# put 1/3 of chips into the validation set
chip_ids = train_meta.chip_id.unique().tolist()
val_chip_ids = random.sample(chip_ids, round(len(chip_ids) * 0.33))
val_mask = train_meta.chip_id.isin(val_chip_ids)
val = train_meta[val_mask].copy().reset_index(drop=True)
train = train_meta[~val_mask].copy().reset_index(drop=True)
# separate features from labels
feature_cols = ["chip_id"] + [f"{band}_path" for band in BANDS]
val_x = val[feature_cols].copy()
val_y = val[["chip_id", "label_path"]].copy()
train_x = train[feature_cols].copy()
train_y = train[["chip_id", "label_path"]].copy()
print((train_x.shape, train_y.shape),(val_x.shape, val_y.shape))

((5335, 5), (5335, 2)) ((2628, 5), (2628, 2))


In [9]:
# Set up pytorch_lightning.Trainer object
cloud_model = CloudModel(
    bands=BANDS,
    x_train=train_x,
    y_train=train_y,
    x_val=val_x,
    y_val=val_y,
    hparams = {
        "backbone":None,
        "weights": None,
        "gpu":False,
        "num_workers":2,
        "batch_size":32,
        "transform":None
    }
)

checkpoint_callback = pl.callbacks.ModelCheckpoint(
    monitor="iou_epoch", mode="max", verbose=True
)

early_stopping_callback = pl.callbacks.early_stopping.EarlyStopping(
    monitor="iou_epoch",
    patience=(cloud_model.patience * 3),
    mode="max",
    verbose=True,
)

trainer = pl.Trainer(
    fast_dev_run=False,
    callbacks=[checkpoint_callback, early_stopping_callback],
    max_epochs = 10
    )

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [10]:
trainer.fit(model=cloud_model)


  | Name  | Type   | Params
---------------------------------
0 | model | UNET38 | 538 K 
---------------------------------
538 K     Trainable params
0         Non-trainable params
538 K     Total params
2.156     Total estimated model params size (MB)


Epoch 0: 100%|██████████| 250/250 [55:58<00:00, 13.38s/it, loss=0.474, v_num=51, loss_step=0.430, iou_step=0.738, iou_epoch=0.692]

Metric iou_epoch improved. New best score: 0.692
Epoch 0, global step 166: iou_epoch reached 0.69186 (best 0.69186), saving model to "j:\CloudCover\lightning_logs\version_51\checkpoints\epoch=0-step=166.ckpt" as top 1


Epoch 1:  40%|████      | 101/250 [27:34<40:17, 16.22s/it, loss=0.416, v_num=51, loss_step=0.407, iou_step=0.738, iou_epoch=0.692, loss_epoch=0.553]

In [ ]:
# # save the model
DATA_DIR = Path.cwd().parent.resolve() / "CloudCover/data"
submission_assets_dir = DATA_DIR / "assets"
submission_assets_dir.mkdir(parents=True, exist_ok=True)

model_weight_path = submission_assets_dir / "cloud_model_unet38.pt"
torch.save(cloud_model.state_dict(), model_weight_path)